In [2]:
import asyncio
import nest_asyncio
import concurrent.futures.process
import numpy as np
from Workspace.BackTest.DataFactory import FactoryManager
import SystemConfig
from typing import List, Dict, Any
from Workspace.BackTest.Storage.StorageCollector import ClosingSyncStorage
import Workspace.Utils.BaseUtils as base_utils

nest_asyncio.apply()

start_date = "2025-01-01"
end_date = "2025-01-5"
symbol = SystemConfig.Streaming.symbols
intervals = SystemConfig.Streaming.intervals# + ["1m"]
storage = ClosingSyncStorage()

ins_factory = FactoryManager(start_date, end_date)

symbols = ["BTCUSDT"]#, "ETHUSDT", "ADAUSDT"]
data = asyncio.run(ins_factory.fetch_multiple_klines(symbols, intervals))

process_data = ins_factory.process_for_analysis(data)
base=process_data["BTCUSDT"]["1m"]
closing_3m = ins_factory.generate_kline_closing_sync(base, process_data["BTCUSDT"]["3m"], "3m")
closing_5m = ins_factory.generate_kline_closing_sync(base, process_data["BTCUSDT"]["5m"], "5m")
base_indices = ins_factory.generate_indices_arange("1m", base)

  📨 BTCUSDT_5m 수신 완료
  📨 BTCUSDT_3m 수신 완료
  📨 BTCUSDT_1m 수신 완료


In [3]:
def generate_indices_by_interval(base_indices, interval:str, lookback_days:int=1):
    index_step = base_utils.get_interval_minutes(interval)
    day_step = base_utils.get_interval_minutes("1d") * lookback_days
    result = []
    for current_index in base_indices:
        start_idx = max(current_index-day_step, 0)
        if start_idx == 0:
            continue
        condition = np.where((base_indices >= start_idx)&
                            (base_indices < current_index)&
                            (base_indices % index_step==0))
        add_indices = np.append(condition, current_index)
        result.append(np.append(condition, current_index))
    return result

In [5]:
indices_1m = generate_indices_by_interval(base_indices, '1m', 1)
indices_3m = generate_indices_by_interval(base_indices, '3m', 1)
indices_5m = generate_indices_by_interval(base_indices, '5m', 1)

In [6]:
print(f" 1m : {len(indices_1m)}")
print(f" 1m : {len(indices_3m)}")
print(f" 1m : {len(indices_5m)}")

 1m : 4320
 1m : 4320
 1m : 4320


In [16]:
base_indices[1]

np.int64(1)

In [33]:
for i, v in enumerate(indices_3m):
    base[v]
    closing_3m[v]
    closing_5m[v]

In [ ]:
from typing import List, Dict, Optional, Any
import numpy as np
import asyncio
import os, sys
import pickle

home_path = os.path.expanduser("~")
sys.path.append(os.path.join(home_path, "github", "Thunder", "Binance"))

path = os.path.join(home_path, "github", "TestData")
-
os.path.isdir(path)

False